In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357238 sha256=77f829b98e85522c04b5b442a0a5223cb887307692adb17a950e751567e91fe3
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [18]:
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise import Dataset, Reader, SVD, SVDpp, accuracy, NMF
from urllib.request import urlretrieve
import pandas as pd
import zipfile
import os

In [4]:

url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
ratings_path = 'ml-latest-small/ratings.csv'

if not os.path.exists('ml-latest-small'):
    urlretrieve(url, 'ml-latest-small.zip')
    with zipfile.ZipFile('ml-latest-small.zip', 'r') as file:
        file.extractall('.')

data_to_read = pd.read_csv(ratings_path)
data_to_read


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [20]:
model_SVD = SVD()
model_SVDpp = SVDpp()
model_NMF = NMF()
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(data_to_read[['userId', 'movieId', 'rating']], reader)

In [21]:
result_SVD = cross_validate(model_SVD, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
trainset, testset = train_test_split(data, test_size=0.2)
model_SVD.fit(trainset)
predict = model_SVD.test(testset)
rmse = accuracy.rmse(predict)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8768  0.8634  0.8836  0.8709  0.8750  0.8739  0.0067  
MAE (testset)     0.6721  0.6648  0.6789  0.6690  0.6715  0.6713  0.0046  
Fit time          1.84    1.51    1.48    1.47    1.47    1.55    0.14    
Test time         0.12    0.30    0.11    0.25    0.13    0.18    0.08    
RMSE: 0.8722


In [22]:
result_SVDpp = cross_validate(model_SVDpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
model_SVDpp.fit(trainset)
predict_pp = model_SVDpp.test(testset)
rmse_pp = accuracy.rmse(predict_pp)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8602  0.8668  0.8608  0.8603  0.8574  0.8611  0.0031  
MAE (testset)     0.6606  0.6642  0.6582  0.6596  0.6567  0.6598  0.0025  
Fit time          91.66   90.55   89.15   90.89   90.25   90.50   0.82    
Test time         12.40   12.35   12.30   12.26   12.20   12.30   0.07    
RMSE: 0.8623


In [23]:
result_NMF = cross_validate(model_NMF, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
model_NMF.fit(trainset)
predict_nmf = model_NMF.test(testset)
rmse_nmf = accuracy.rmse(predict_nmf)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9280  0.9139  0.9247  0.9282  0.9251  0.9240  0.0053  
MAE (testset)     0.7107  0.6996  0.7092  0.7088  0.7085  0.7074  0.0040  
Fit time          3.23    2.73    2.73    3.38    2.92    3.00    0.27    
Test time         0.24    0.12    0.24    0.20    0.25    0.21    0.05    
RMSE: 0.9155
